<a href="https://colab.research.google.com/github/gremlin97/EVA-8/blob/main/S5_Norm/EVA3_S5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from model import Net

In [2]:
!pip install torchsummary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from torchsummary import summary
norm = 'GN'

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net(norm,num=2).to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
           Dropout-2           [-1, 16, 26, 26]               0
            Conv2d-3           [-1, 32, 24, 24]           4,608
         MaxPool2d-4           [-1, 32, 12, 12]               0
           Dropout-5           [-1, 32, 12, 12]               0
            Conv2d-6           [-1, 10, 12, 12]             320
            Conv2d-7           [-1, 16, 10, 10]           1,440
           Dropout-8           [-1, 16, 10, 10]               0
            Conv2d-9             [-1, 16, 8, 8]           2,304
          Dropout-10             [-1, 16, 8, 8]               0
           Conv2d-11             [-1, 16, 6, 6]           2,304
AdaptiveAvgPool2d-12             [-1, 16, 1, 1]               0
           Linear-13                   [-1, 10]             170
Total params: 11,290
Trainable params: 

/content/model.py:82: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [4]:
torch.manual_seed(1)
batch_size = 32

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomRotation((-1.0, 1.0), fill=(1,)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.RandomRotation((-1.0, 1.0), fill=(1,)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    correct = 0
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx} Train Accuracy={100. * correct / len(train_loader.dataset)}')

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [5]:
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9)
scheduler =  optim.lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.3)

for epoch in range(20):
    print("Epoch: ",epoch+1)
    train(model, device, train_loader, optimizer, epoch)
    scheduler.step()
    test(model, device, test_loader)

Epoch:  1


loss=0.012216417118906975 batch_id=1874 Train Accuracy=94.21166666666667: 100%|██████████| 1875/1875 [00:34<00:00, 53.84it/s]



Test set: Average loss: 0.0438, Accuracy: 9856/10000 (99%)

Epoch:  2


loss=0.05764397606253624 batch_id=1874 Train Accuracy=97.82: 100%|██████████| 1875/1875 [00:31<00:00, 59.46it/s]



Test set: Average loss: 0.0407, Accuracy: 9874/10000 (99%)

Epoch:  3


...


Test set: Average loss: 0.0198, Accuracy: 9934/10000 (99%)

Epoch:  19


loss=0.003052795073017478 batch_id=1874 Train Accuracy=99.40166666666667: 100%|██████████| 1875/1875 [00:31<00:00, 58.87it/s]



Test set: Average loss: 0.0188, Accuracy: 9940/10000 (99%)

Epoch:  20


loss=0.0018444156739860773 batch_id=1874 Train Accuracy=99.465: 100%|██████████| 1875/1875 [00:31<00:00, 60.03it/s]



Test set: Average loss: 0.0186, Accuracy: 9938/10000 (99%)



In [6]:
from torchsummary import summary
norm = 'LN'

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
modelln = Net(norm).to(device)
summary(modelln, input_size=(1, 28, 28))

torch.manual_seed(1)
batch_size = 32

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomRotation((-1.0, 1.0), fill=(1,)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.RandomRotation((-1.0, 1.0), fill=(1,)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
           Dropout-2           [-1, 16, 26, 26]               0
            Conv2d-3           [-1, 32, 24, 24]           4,608
         MaxPool2d-4           [-1, 32, 12, 12]               0
           Dropout-5           [-1, 32, 12, 12]               0
            Conv2d-6           [-1, 10, 12, 12]             320
            Conv2d-7           [-1, 16, 10, 10]           1,440
           Dropout-8           [-1, 16, 10, 10]               0
            Conv2d-9             [-1, 16, 8, 8]           2,304
          Dropout-10             [-1, 16, 8, 8]               0
           Conv2d-11             [-1, 16, 6, 6]           2,304
AdaptiveAvgPool2d-12             [-1, 16, 1, 1]               0
           Linear-13                   [-1, 10]             170
Total params: 11,290
Trainable params: 

In [7]:
optimizer = optim.SGD(modelln.parameters(), lr=0.05, momentum=0.9)
scheduler =  optim.lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.3)

for epoch in range(20):
    print("Epoch: ",epoch+1)
    train(modelln, device, train_loader, optimizer, epoch)
    scheduler.step()
    test(modelln, device, test_loader)

Epoch:  1


loss=0.035572875291109085 batch_id=1874 Train Accuracy=92.53166666666667: 100%|██████████| 1875/1875 [00:30<00:00, 61.62it/s]



Test set: Average loss: 0.0552, Accuracy: 9832/10000 (98%)

Epoch:  2


loss=0.037841737270355225 batch_id=1874 Train Accuracy=97.59833333333333: 100%|██████████| 1875/1875 [00:30<00:00, 61.75it/s]



Test set: Average loss: 0.0503, Accuracy: 9841/10000 (98%)

Epoch:  3


...


Test set: Average loss: 0.0208, Accuracy: 9933/10000 (99%)

Epoch:  19


loss=0.006920449901372194 batch_id=1874 Train Accuracy=99.39666666666666: 100%|██████████| 1875/1875 [00:30<00:00, 60.95it/s]



Test set: Average loss: 0.0196, Accuracy: 9939/10000 (99%)

Epoch:  20


loss=0.010398844256997108 batch_id=1874 Train Accuracy=99.35: 100%|██████████| 1875/1875 [00:31<00:00, 59.55it/s]



Test set: Average loss: 0.0200, Accuracy: 9934/10000 (99%)



In [8]:
from torchsummary import summary
norm = 'BN'
dev = 'cuda:0'

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
modelbn = Net(norm).to(device)
summary(modelbn, input_size=(1, 28, 28))

torch.manual_seed(1)
batch_size = 32

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomRotation((-1.0, 1.0), fill=(1,)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.RandomRotation((-1.0, 1.0), fill=(1,)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
       BatchNorm2d-2           [-1, 16, 26, 26]              32
           Dropout-3           [-1, 16, 26, 26]               0
            Conv2d-4           [-1, 32, 24, 24]           4,608
       BatchNorm2d-5           [-1, 32, 24, 24]              64
         MaxPool2d-6           [-1, 32, 12, 12]               0
           Dropout-7           [-1, 32, 12, 12]               0
            Conv2d-8           [-1, 10, 12, 12]             320
       BatchNorm2d-9           [-1, 10, 12, 12]              20
           Conv2d-10           [-1, 16, 10, 10]           1,440
      BatchNorm2d-11           [-1, 16, 10, 10]              32
          Dropout-12           [-1, 16, 10, 10]               0
           Conv2d-13             [-1, 16, 8, 8]           2,304
      BatchNorm2d-14             [-1, 1

In [9]:
optimizer = optim.SGD(modelbn.parameters(), lr=0.05, momentum=0.9)
scheduler =  optim.lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.3)

for epoch in range(20):
    print("Epoch: ",epoch+1)
    train(modelbn, device, train_loader, optimizer, epoch)
    scheduler.step()
    test(modelbn, device, test_loader)

Epoch:  1


loss=0.015754293650388718 batch_id=1874 Train Accuracy=94.815: 100%|██████████| 1875/1875 [00:30<00:00, 60.75it/s]



Test set: Average loss: 0.0491, Accuracy: 9831/10000 (98%)

Epoch:  2


loss=0.0910787582397461 batch_id=1874 Train Accuracy=97.61333333333333: 100%|██████████| 1875/1875 [00:30<00:00, 60.84it/s]



Test set: Average loss: 0.0624, Accuracy: 9798/10000 (98%)

Epoch:  3


...


Test set: Average loss: 0.0189, Accuracy: 9938/10000 (99%)

Epoch:  19


loss=0.0038079076912254095 batch_id=1874 Train Accuracy=99.43166666666667: 100%|██████████| 1875/1875 [00:31<00:00, 58.64it/s]



Test set: Average loss: 0.0183, Accuracy: 9939/10000 (99%)

Epoch:  20


loss=0.01800128072500229 batch_id=1874 Train Accuracy=99.39166666666667: 100%|██████████| 1875/1875 [00:31<00:00, 58.90it/s]



Test set: Average loss: 0.0188, Accuracy: 9939/10000 (99%)



In [10]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torchvision import datasets, transforms

# class Net1(nn.Module):
#     def __init__(self, norm='GN', num=1, device='cuda:0'):
#         super(Net1, self).__init__()
#         self.norm = norm
#         self.num = num
#         self.device = device
#         self.conv1 = nn.Conv2d(1, 16, 3, padding=0, bias=False) # RF:1+(3-1)1=3; ji=1,jo=1; 28x28x1 -> 26x26x16
#         self.bn1 = nn.BatchNorm2d(16)
#         self.drop1 = nn.Dropout(0.1)
#         self.conv3 = nn.Conv2d(16, 32, 3, padding=0, bias=False) # RF:4+(3-1)2=8; ji=2,jo=2; 26x26x16 -> 24x24x16
#         self.bn2 = nn.BatchNorm2d(32)
#         self.pool2 = nn.MaxPool2d(2, 2) # RF:8+(2-1)2=10; ji=2,jo=4; 24x24x16 -> 12x12x16
#         self.drop2 = nn.Dropout(0.1)
#         self.conv5 = nn.Conv2d(32, 10, 1, padding=0, bias=False) # RF:10+(3-1)4=18; ji=4,jo=4; 12x12x16 -> 10x10x16
#         self.bn3 = nn.BatchNorm2d(10) 
#         self.drop3 = nn.Dropout(0.1)
#         self.conv6 = nn.Conv2d(10, 16, 3, padding=0, bias=False) # RF:18+(2-1)4=22; ji=4,jo=8; 10x10x16 -> 16x16x16
#         self.bn4 = nn.BatchNorm2d(16)
#         self.drop4 = nn.Dropout(0.1)
#         self.conv10 = nn.Conv2d(16, 16, 3, padding=0, bias=False) # RF:18+(2-1)4=22; ji=4,jo=8; 16x16x16 -> 14x14x16
#         self.bn10 = nn.BatchNorm2d(16)
#         self.drop10 = nn.Dropout(0.1)
#         self.conv7 = nn.Conv2d(16, 16, 3, padding=0, bias=False) # RF:18+(2-1)4=22; ji=4,jo=8; 5x5x16 -> 3x3x16
#         self.gap = nn.AdaptiveAvgPool2d((1,1)) 

#         self.lin = nn.Linear(16, 10)

#     def forward(self, x):

#         x = F.relu(self.conv1(x))

#         if self.norm == 'BN':
#           n_chans = x.shape[1]
#           running_mean = torch.zeros(n_chans).to(self.device) 
#           running_std = torch.ones(n_chans).to(self.device)
#           print(self.device)
#           print(running_mean)
#           x = F.batch_norm(x, running_mean, running_std, training=True)
#           x = F.relu(self.conv3(self.drop1(x)))
#           n_chans = x.shape[1]
#           running_mean = torch.zeros(n_chans).to(self.device) 
#           running_std = torch.ones(n_chans).to(self.device) 
#           x = F.batch_norm(x, running_mean, running_std, training=True)
#           x = F.relu(self.conv5(self.drop2(self.pool2(x))))
#           n_chans = x.shape[1]
#           running_mean = torch.zeros(n_chans).to(self.device) 
#           running_std = torch.ones(n_chans).to(self.device) 
#           x = F.batch_norm(x, running_mean, running_std, training=True)
#           x = F.relu(self.conv6(x))
#           n_chans = x.shape[1]
#           running_mean = torch.zeros(n_chans).to(self.device)  
#           running_std = torch.ones(n_chans).to(self.device) 
#           x = F.batch_norm(x, running_mean, running_std, training=True)
#           x = F.relu(self.conv10(self.drop4(x)))
#           n_chans = x.shape[1]
#           running_mean = torch.zeros(n_chans).to(self.device)  
#           running_std = torch.ones(n_chans).to(self.device) 
#           x = F.batch_norm(x, running_mean, running_std, training=True)

#         elif self.norm == 'GN':
#           x = F.group_norm(x,self.num)
#           x = F.relu(self.conv3(self.drop1(x)))
#           x = F.group_norm(x, self.num)
#           x = F.relu(self.conv5(self.drop2(self.pool2(x))))
#           x = F.group_norm(x, self.num)
#           x = F.relu(self.conv6(x))
#           x = F.group_norm(x, self.num)
#           x = F.relu(self.conv10(self.drop4(x)))
#           x = F.group_norm(x, self.num)

#         else:
#           x = F.layer_norm(x, [16, 26, 26])
#           x = F.relu(self.conv3(self.drop1(x)))
#           x = F.layer_norm(x, [32, 24, 24])
#           x = F.relu(self.conv5(self.drop2(self.pool2(x))))
#           x = F.layer_norm(x, [10, 12, 12])
#           x = F.relu(self.conv6(x))
#           x = F.layer_norm(x, [16, 10, 10])
#           x = F.relu(self.conv10(self.drop4(x)))
#           x = F.layer_norm(x, [16, 8, 8])

#         x = self.conv7(self.drop10(x))

#         x = self.gap(x)

#         x = x.view(-1, 16)

#         x = self.lin(x)
        
#         return F.log_softmax(x)
